# Tutorial 2. Classifier-free Guidance

## 1 Introduction

In this tutorial, we'll explore how to customize a classifier-free guidance (CFG) model for a specific task. Let's first review how CFG works.

### 1.1 Classifier-free Guidance
We consider a conditional generation task, where we want to sample from a conditional distribution $q_0(\bm x|\bm y)$. The score function can be written as

$$
\begin{equation}
\nabla_{\bm x}\log q_t(\bm x_t|\bm y)=\nabla_{\bm x}\log q_t(\bm x_t) + \nabla_{\bm x}\log q_t(\bm y|\bm x_t),
\end{equation}
$$

where the first term on the right side is the score function of unconditional distribution $q_t(\bm x_t)$ that can be estimated by training an unconditional diffusion model on the dataset. The second term is what the guidance methods need to estimate. CFG uses $\nabla_{\bm x}\log q_t(\bm y|\bm x_t)=\nabla_{\bm x}\log q_t(\bm x_t|\bm y)-\nabla_{\bm x}\log q_t(\bm x_t)$. By training a conditional noise prediction model $\bm\epsilon_\theta(\bm x_t, t, \bm y)$, the sampling process can be guided with no additional classifier:

$$
\begin{equation}
\bar{\bm\epsilon_\theta}(\bm x_t, t, \bm y)=\bm\epsilon_\theta(\bm x_t, t)-w\cdot\left(\bm\epsilon_\theta(\bm x_t, t, \bm y)-\bm\epsilon_\theta(\bm x_t, t)\right),
\end{equation}
$$

where $w$ is the guidance strength. In practice, we use a dummy condition $\bm y=\bm\Phi$ to represent unconditional generation, i.e., $\bm\epsilon_\theta(\bm x_t, t,\bm\Phi)=\bm\epsilon_\theta(\bm x_t, t)$.

In decision-making, the condition $\bm y$ may be highly complex multi-modal data, e.g., image-based observations, language instructions, point clouds, and so on. Some works even use large transformers for multimodal fusion processing of conditions, while using small MLPs as the diffusion NN backbone. Therefore, in CleanDiffuser, we believe it is forward-looking and necessary to decouple the neural networks of Diffusion $\bm\epsilon_\theta$ and the conditions $\bm\zeta_\phi$ to facilitate development and debugging. The conditional diffusion models in CleanDiffuser are actually implemented as $\epsilon_\theta(\bm x_t, t, \bm\zeta_\phi(\bm y))$, and the dummy condition is defined to be zeros $\bm\zeta_\phi(\bm\Phi)=\bm 0$ without loss of generality. This is why in tutorial 1, we need both a `NNDiffusion` and a `NNCondition` to create a diffusion model. The `NNDiffusion` is the $\bm\epsilon_\theta$ here and the `NNCondition` is the $\bm\zeta_\phi$.

### 1.2 Diffusion Planners

In this tutorial, we'll implement a diffusion planner using CFG. The basic idea of diffusion planners is to generate high-performance decision trajectories and extract the first action in the trajectory to execute. This is actually very similar to MPC and many planning-based model-based RL algorithms. They use searching methods and dynamic models to obtain high-performance trajectories, while diffusion planners use conditional generation to achieve this.

Obviously, we need a "high-performance" variable as a condition to guide the generation. A simple and commonly used method is to use the discounted return-to-go of trajectories $\sum_{s=t}^T \gamma^{s-t} r_s$ in the dataset as the condition. It is actually a Monte Carlo estimation of the value of the trajectory. During training, we normalize the values in the dataset to the range [0, 1], so that a value of 1 represents the highest performance. During inference, we use relatively high normalized values like 0.8-1.0 as conditions to generate high-performance trajectories. For more details, we recommend reading [Diffuser](https://arxiv.org/abs/2205.09991) and [Decision Diffuser](https://openreview.net/forum?id=sP1fo2K9DFG).


## 2 Setting up the Environment and Preparing the Dataset

We use D4RL-MuJoCo-halfcheetah-medium-expert-v2 as the benchmark. D4RL-MuJoCo is a widely used offline RL benchmark. `halfcheetah-medium-expert-v2` requires to control a halfcheetah robot to move forward as fast as possible, and it provides a medium-expert-quality demonstration dataset.

In [2]:
import gym
import d4rl
from cleandiffuser.dataset.d4rl_mujoco_dataset import D4RLMuJoCoDataset


# horizon=4 is enough for halfcheetah tasks as mentioned in Diffuser paper.
horizon = 4
env = gym.make("halfcheetah-medium-expert-v2")
dataset = D4RLMuJoCoDataset(env.get_dataset(), terminal_penalty=-100, horizon=horizon)
obs_dim, act_dim = dataset.o_dim, dataset.a_dim

No module named 'flow'
/home/dzb/miniforge3/envs/cleandiffuser/lib/python3.9/site-packages/glfw/__init__.py:914: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
No module named 'carla'
pybullet build time: Nov 28 2023 23:52:03
/home/dzb/miniforge3/envs/cleandiffuser/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
load datafile: 100%|██████████| 9/9 [00:05<00:00,  1.54it/s]


## 3 Customizing a CFG Model

To customize a CFG model, we are actually designing the condition network $\bm\zeta_\phi$. So our first step is to check how the diffusion network uses the condition embedding. Suppose we use `DiT1d` as the diffusion network and we take a look at the forward function of `DiT1d` to see it takes a tensor of shape `(batch_size, embedding_dim)` as the condition embedding. And we use the value tensor of shape `(batch_size, 1)` as the generation condition. So we need to design a condition network that can map the value tensor to the condition embedding tensor. Here we use a simple MLP as the condition network (See `ValueNNCondition` below). Then we can simply combine the `DiT1d` and `ValueNNCondition` to create a diffusion model as we did in tutorial 1.

In [3]:
from cleandiffuser.nn_condition import BaseNNCondition, get_mask
from cleandiffuser.utils import at_least_ndim
from cleandiffuser.nn_diffusion import DiT1d
from cleandiffuser.diffusion import ContinuousDiffusionSDE
import torch
import torch.nn as nn


class ValueNNCondition(BaseNNCondition):
    """ Simple MLP NNCondition for value conditioning.
    
    value (bs, 1) -> ValueNNCondition -> embedding (bs, emb_dim)

    Args:
        emb_dim (int): Embedding dimension.
        dropout (float): Label dropout rate.
    
    Example:
        >>> value = torch.rand(32, 1)
        >>> condition = ValueNNCondition(emb_dim=64, dropout=0.25)
        >>> # If condition.training, embedding will be masked to be dummy condition 
        >>> # with label dropout rate 0.25.
        >>> embedding = condition(value) 
        >>> embedding.shape
        torch.Size([32, 64])
    """
    def __init__(self, emb_dim: int, dropout: float = 0.25):
        super().__init__()
        self.dropout = dropout
        self.mlp = nn.Sequential(
            nn.Linear(1, 256), nn.SiLU(),
            nn.Linear(256, 256), nn.SiLU(),
            nn.Linear(256, emb_dim))
    def forward(self, condition: torch.Tensor, mask: torch.Tensor = None):
        mask = get_mask(
            mask, (condition.shape[0],), self.dropout, self.training, condition.device)
        mask = at_least_ndim(mask, condition.dim())
        return condition * mask
    

device = "cuda:0" if torch.cuda.is_available() else "cpu"

nn_diffusion = DiT1d(
    obs_dim + act_dim, emb_dim=128, d_model=320, n_heads=10, depth=2, 
    timestep_emb_type="untrainable_fourier")
nn_condition = ValueNNCondition(emb_dim=128, dropout=0.25)

fix_mask = torch.zeros((horizon, obs_dim + act_dim))
fix_mask[0, :obs_dim] = 1.
loss_weight = torch.ones((horizon, obs_dim + act_dim))
loss_weight[0, obs_dim:] = 10.

planner = ContinuousDiffusionSDE(
    nn_diffusion=nn_diffusion, nn_condition=nn_condition,
    fix_mask=fix_mask, loss_weight=loss_weight, ema_rate=0.9999,
    device=device)

random_obs = torch.randn((obs_dim,))
prior = torch.zeros((1, horizon, obs_dim + act_dim))
prior[:, 0, :obs_dim] = random_obs[None, :]

traj, log = planner.sample(
    prior, solver="ddpm", n_samples=1, sample_steps=5)

print(f'Trajectory shape: {traj.shape}')
print(f'First observation MSE: {(traj[0, 0, :obs_dim].cpu() - random_obs).pow(2).mean()}')

Trajectory shape: torch.Size([1, 4, 23])
First observation MSE: 0.0


You may notice that we use some new variables like `fix_mask` and `loss_weight` that we didn't use in tutorial 1. Let's explain them here. 

The diffusion-generated trajectories are looks like:
$$
\bm\tau = \left[
\begin{aligned}
&\bm s_0, \bm s_1, \cdots, \bm s_{H-1} \\
&\bm a_0, \bm a_1, \cdots, \bm a_{H-1} \\
\end{aligned}
\right],
$$
where $\bm s_0$ is the current state and it is known and fixed during generation. So the generation process works like an image inpainting task. `fix_mask` is a tensor with the same shape as $\bm\tau$ and it is 1 for known items and 0 for unknown items. During training, the fixed parts are maintained and not contributed to the loss. During inference, the fixed parts in `prior` are used to do inpainting. This is why we set `prior[:, 0, :obs_dim] = obs` before sampling.

`loss_weight` is also a tensor with the same shape as $\bm\tau$ and it is used to weight the loss. In this tutorial, since the first action $\bm a_0$ directly affects the decision-making performance, we set the weight of the first action to be 10 times larger than the other parts.

## 4 Training the Diffusion Model

This part is almost the same as tutorial 1, except that the generated data is the trajectory and the generation condition is the value. You may find it strange that we divide the value tensor by 1200. The 1200 is actually an empirical value that makes the value tensor in the range [0, 1], and is observed in the dataset. It's may be a little bit dumb, but it is simple and works well.

In [5]:
import os

from torch.utils.data import DataLoader

from cleandiffuser.utils import loop_dataloader


savepath = "../tutorials/results/2_classifier_free_guidance/"
if not os.path.exists(savepath):
    os.makedirs(savepath)

dataloader = DataLoader(
    dataset, batch_size=64, shuffle=True, num_workers=4, persistent_workers=True)

n_gradient_steps = 0
avg_loss = 0.
planner.train()
for batch in loop_dataloader(dataloader):
    
    obs, act = batch["obs"]["state"].to(device), batch["act"].to(device)
    val = batch["val"].to(device) / 1200.
    x0 = torch.cat([obs, act], dim=-1)

    avg_loss += planner.update(x0=x0, condition=val)["loss"]
    
    n_gradient_steps += 1
    
    if n_gradient_steps % 1000 == 0:
        print(f'Step: {n_gradient_steps} | Loss: {avg_loss / 1000}')
        avg_loss = 0.
    
    if n_gradient_steps % 100_000 == 0:
        planner.save(savepath + "diffusion.pt")
    
    if n_gradient_steps == 500_000:
        break
    

## 5 Evaluation

Let's see how our customized CFG planner performs in `halfcheetah-medium-expert-v2`! We parallelly interact with 50 environments and use 3 random seeds to evaluate the performance. The evaluation metric is the normalized episode return, with 100 being the expert-performance and 0 being the random-performance. We use DDPM with 5 sampling steps (compared to 100 sampling steps used in Decision Diffuser official implementation) to generate trajectories. The results show that we can achieve a D4RL score of 88.4. For a model without carefully tuning, this is not bad!

In [8]:
import numpy as np


solver = "ddpm"
sampling_step = 5
num_episodes = 3
num_envs = 50
target_return = 0.95
w_cfg = 1.2

planner.load(savepath + "diffusion.pt")
planner.eval()

# Parallelize evaluation
env_eval = gym.vector.make('halfcheetah-medium-expert-v2', num_envs=num_envs)

# Get normalizers
normalizer = dataset.get_normalizer()

episode_rewards = []

prior = torch.zeros((num_envs, horizon, obs_dim + act_dim), device=device)
condition = torch.ones((num_envs, 1), device=device) * target_return
for i in range(num_episodes):

    obs, ep_reward, cum_done, t = env_eval.reset(), 0., 0., 0

    while not np.all(cum_done) and t < 1000 + 1:
        
        # normalize obs
        obs = torch.tensor(normalizer.normalize(obs), device=device, dtype=torch.float32)

        # sample trajectories
        prior[:, 0, :obs_dim] = obs
        traj, log = planner.sample(
            prior, 
            solver=solver,
            n_samples=num_envs, 
            sample_step_schedule="quad_continuous",
            sample_steps=sampling_step, use_ema=True,
            condition_cfg=condition, w_cfg=w_cfg, temperature=1.0)
        act = traj[:, 0, obs_dim:].clip(-1., 1.).cpu().numpy()

        # step
        obs, rew, done, info = env_eval.step(act)

        t += 1
        cum_done = done if cum_done is None else np.logical_or(cum_done, done)
        ep_reward += (rew * (1 - cum_done)) if t < 1000 else rew
        print(f'[t={t}] rew: {np.around((rew * (1 - cum_done)), 2)}')

    episode_rewards.append(ep_reward)

episode_rewards = [list(map(lambda x: env.get_normalized_score(x), r)) for r in episode_rewards]
episode_rewards = np.array(episode_rewards)
mean_rewards = np.mean(episode_rewards, -1) * 100.
print(f'D4RL score: {mean_rewards.mean():.3f} +- {mean_rewards.std():.3f}')

/home/dzb/miniforge3/envs/cleandiffuser/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


[t=1] rew: [-0.48 -0.39 -0.7  -0.7  -0.39 -0.47 -0.31 -0.57 -0.83 -0.67 -0.69 -0.73
 -0.57 -0.72 -0.27 -0.59 -0.39 -0.32 -0.8  -0.58 -0.32 -0.75 -0.52 -0.74
 -0.29 -0.73 -0.27 -0.74 -0.53 -0.77 -0.53 -0.45 -0.36 -0.5  -0.87 -0.51
 -0.49 -0.96 -0.68 -0.57 -1.09 -0.85 -0.38 -0.63 -0.56 -0.99 -0.61 -0.83
 -0.39 -0.76]
[t=2] rew: [-0.55 -0.51 -0.94 -0.67 -0.87 -0.77 -0.56 -0.89 -0.78 -0.78 -0.46 -0.62
 -0.58 -0.75 -0.65 -0.67 -0.63 -0.94 -1.02 -0.75 -0.58 -0.93 -0.89 -0.89
 -0.48 -1.09 -0.32 -0.91 -0.45 -0.71 -0.82 -0.61 -0.42 -0.39 -0.94 -0.53
 -0.74 -0.86 -0.63 -0.6  -1.09 -0.82 -0.62 -0.81 -0.55 -1.08 -0.57 -0.83
 -0.71 -0.65]
[t=3] rew: [-0.19 -0.13 -0.51 -0.13 -0.54 -0.59 -0.34 -0.24 -0.26 -0.33  0.02  0.13
 -0.13 -0.14 -0.43 -0.07 -0.51 -0.56 -0.44 -0.4  -0.26 -0.4  -0.43 -0.03
 -0.16 -0.63 -0.03 -0.37  0.12 -0.15 -0.33 -0.3   0.01 -0.05 -0.1   0.01
 -0.37 -0.06  0.28  0.22 -0.26 -0.14 -0.32 -0.61 -0.45 -0.77 -0.11 -0.26
  0.06 -0.18]
[t=4] rew: [ 0.13  0.1  -0.15  0.24  0.32 -0.23  